In [83]:
import pandas as pd
from pathlib import Path
import seaborn as sns

if not Path("speeches.csv.zip").exists():
    from os import system
    system("wget https://huggingface.co/datasets/5roop/parlasent_data/resolve/main/speeches.csv.zip")
df = pd.read_csv("speeches.csv.zip")

/tmp/ipykernel_2967799/2083943282.py:8: DtypeWarning: Columns (10,11,12,13,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("speeches.csv.zip")


In [84]:
# Filtering
# Select only SI speeches:
c0 = df.country == "HR"

# Keep only MPs
c1 = (df.Speaker_MP == "MP")# & (df.Speaker_minister == "notMinister")

# Keep only speeches where speaker is either Opposition or Coalition:
c2 = df.Party_status.isin(["Opposition", "Coalition"])

# Keep only people that have at least 10 speeches:
gb = df[c0&c1&c2].groupby("Speaker_name").logits_pondered.count().reset_index()
speakers_to_keep = gb.Speaker_name[gb.logits_pondered >= 10]
c3 = df.Speaker_name.isin(speakers_to_keep)


ndf = df[c0&c1&c2&c3]
print(f"Available terms: \n{ndf.Term.value_counts()}")

Available terms: 
Term
9. mandat     112020
7. mandat      91920
5. mandat      73334
10. mandat     72935
6. mandat      67992
8. mandat      10317
Name: count, dtype: int64


In [85]:
# Selecting the term:
c0 = ndf.Term == "9. mandat"
nndf = ndf[c0].reset_index(drop=True)

In [86]:
gb1 = nndf.groupby(["Speaker_name", "Party_status"]).agg({
    "logits_pondered": ["mean", "count"]
}).reset_index()
gb1 = gb1.set_axis(gb1.columns.map(lambda l: '_'.join(l).rstrip("_").replace("logits_pondered_", "")), axis=1,).pivot(
    index="Speaker_name",
    columns="Party_status",
    values=["mean", "count"]
).reset_index()
gb1 = gb1.set_axis(gb1.columns.map(lambda l: '_'.join(l).rstrip("_").replace("logits_pondered_", "")), axis=1,)
gb2 = nndf.groupby(["Speaker_name"]).agg({
    "logits_pondered": ["mean", "count"]
}).reset_index()
gb2.columns = "Speaker_name mean count".split()

gb2 = gb2.merge(gb1, on="Speaker_name", how="left")
gb2 = gb2.sort_values(by="mean", ascending=True)
gb2

,Speaker_name,mean,count,mean_Coalition,mean_Opposition,count_Coalition,count_Opposition
104,"Pernar, Ivan",0.945321,1799,0.945321,NaN,1799.0,NaN
23,"Culej, Stevo",1.090739,382,1.090739,NaN,382.0,NaN
52,"Klarin, Ivan",1.110596,37,1.110596,NaN,37.0,NaN
20,"Bunjac, Branimir",1.166203,1441,1.166203,NaN,1441.0,NaN
69,"Lenart, Željko",1.182931,446,1.182931,NaN,446.0,NaN
...,...,...,...,...,...,...,...
18,"Brkić, Milijan",3.650599,11477,3.650599,NaN,11477.0,NaN
157,"Štromar, Predrag",3.710791,48,3.710791,NaN,48.0,NaN
57,"Kopić, Vlatko",3.746779,35,3.746779,NaN,35.0,NaN
4,"Balić, Marijana",3.868353,41,3.868353,NaN,41.0,NaN


In [87]:
gb2

,Speaker_name,mean,count,mean_Coalition,mean_Opposition,count_Coalition,count_Opposition
104,"Pernar, Ivan",0.945321,1799,0.945321,NaN,1799.0,NaN
23,"Culej, Stevo",1.090739,382,1.090739,NaN,382.0,NaN
52,"Klarin, Ivan",1.110596,37,1.110596,NaN,37.0,NaN
20,"Bunjac, Branimir",1.166203,1441,1.166203,NaN,1441.0,NaN
69,"Lenart, Željko",1.182931,446,1.182931,NaN,446.0,NaN
...,...,...,...,...,...,...,...
18,"Brkić, Milijan",3.650599,11477,3.650599,NaN,11477.0,NaN
157,"Štromar, Predrag",3.710791,48,3.710791,NaN,48.0,NaN
57,"Kopić, Vlatko",3.746779,35,3.746779,NaN,35.0,NaN
4,"Balić, Marijana",3.868353,41,3.868353,NaN,41.0,NaN


In [88]:
gb1.reset_index()

,index,Speaker_name,mean_Coalition,mean_Opposition,count_Coalition,count_Opposition
0,0,"Aleksić, Goran",1.896848,NaN,644.0,NaN
1,1,"Ambrušec, Ljubica",NaN,3.505083,NaN,20.0
2,2,"Babić, Ante",2.483455,NaN,699.0,NaN
3,3,"Babić, Vedran",2.208034,NaN,79.0,NaN
4,4,"Balić, Marijana",3.868353,NaN,41.0,NaN
...,...,...,...,...,...,...
154,154,"Šimić, Miroslav",NaN,2.783642,NaN,147.0
155,155,"Šipić, Ivan",2.701190,NaN,107.0,NaN
156,156,"Škorić, Petar",2.222604,NaN,200.0,NaN
157,157,"Štromar, Predrag",3.710791,NaN,48.0,NaN


In [89]:
gb2.columns.map("_".join).map(lambda s: s.replace("logits_pondered", "_"))

Index(['S_p_e_a_k_e_r___n_a_m_e', 'm_e_a_n', 'c_o_u_n_t',
       'm_e_a_n___C_o_a_l_i_t_i_o_n', 'm_e_a_n___O_p_p_o_s_i_t_i_o_n',
       'c_o_u_n_t___C_o_a_l_i_t_i_o_n', 'c_o_u_n_t___O_p_p_o_s_i_t_i_o_n'],
      dtype='object')

In [90]:
gb2.columns.get_level_values(0)

Index(['Speaker_name', 'mean', 'count', 'mean_Coalition', 'mean_Opposition',
       'count_Coalition', 'count_Opposition'],
      dtype='object')